# [Word2Vec: treinamento de Word Embedding](https://cursos.alura.com.br/course/word2vec-treinamento-word-embedding)

## Datasets

* [Site NILC USP com Word Embeddings](http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc)

* [Arquivo treino](https://caelum-online-public.s3.amazonaws.com/1638-word-embedding/treino.csv)

* [Arquivo teste](https://caelum-online-public.s3.amazonaws.com/1638-word-embedding/teste.csv)



In [ ]:
# Abaixo as versões utilizadas no google colab
!pip list -v

Package                       Version               Location                                Installer
----------------------------- --------------------- --------------------------------------- ---------
absl-py                       1.4.0                 /usr/local/lib/python3.10/dist-packages pip
alabaster                     0.7.13                /usr/local/lib/python3.10/dist-packages pip
albumentations                1.2.1                 /usr/local/lib/python3.10/dist-packages pip
altair                        4.2.2                 /usr/local/lib/python3.10/dist-packages pip
anyio                         3.6.2                 /usr/local/lib/python3.10/dist-packages pip
appdirs                       1.4.4                 /usr/local/lib/python3.10/dist-packages pip
argon2-cffi                   21.3.0                /usr/local/lib/python3.10/dist-packages pip
argon2-cffi-bindings          21.2.0                /usr/local/lib/python3.10/dist-packages pip
array-record                

In [1]:
import pandas as pd

df_treino = pd.read_csv("/content/drive/MyDrive/ml-dataset/word2vec2/treino.csv")
df_teste = pd.read_csv("/content/drive/MyDrive/ml-dataset/word2vec2/teste.csv")

In [2]:
df_treino.sample(5)

,title,text,date,category,subcategory,link
14774,Trump defende punir quem queima bandeira ameri...,"Qual direito é maior: a bandeira americana, sí...",2016-11-30,mundo,NaN,http://www1.folha.uol.com.br/mundo/2016/11/183...
67192,"Durante o terremoto, grupo de brasileiros fazi...",Quando o terremoto de magnitude 7.8 atingiu o ...,2015-04-25,mundo,NaN,http://www1.folha.uol.com.br/mundo/2015/04/162...
39003,Tite enumera estatísticas para explicar eficiê...,São 21 gols marcados em 12 jogos e apenas três...,2016-03-27,esporte,NaN,http://www1.folha.uol.com.br/esporte/2016/03/1...
87823,Bill Gates lidera fundo de US$ 1 bilhão para i...,O bilionário e filantropo americano Bill Gates...,2016-12-13,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/12/1...
11000,Neymar e Cavani se desentendem por cobranças d...,Neymar e Cavani se desentenderam duas vezes du...,2017-09-17,esporte,NaN,http://www1.folha.uol.com.br/esporte/2017/09/1...


In [3]:
!python -m spacy download pt_core_news_sm

2023-06-11 15:33:49.441620: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 15:33:51.192297: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 78.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [4]:
!ls

drive  sample_data


In [5]:
import spacy

nlp = spacy.load("pt_core_news_sm")

In [6]:
texto = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(texto)

In [7]:
type(doc)

spacy.tokens.doc.Doc

In [8]:
doc[0]

Rio

In [9]:
type(doc[0])

spacy.tokens.token.Token

In [10]:
doc.ents

(Rio de Janeiro,)

In [11]:
doc[0].is_stop

False

In [12]:
doc[1].is_stop

True

In [13]:
textos_para_tratamento = (titulo.lower() for titulo in df_treino.title)

In [14]:
def trata_textos(doc):
  tokens = []
  for token in doc:
    eh_valido = not token.is_stop and token.is_alpha
    if eh_valido:
      tokens.append(token.text)
  if len(tokens) > 2:
    return " ".join(tokens)
  return None

In [15]:
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [16]:
%%time
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, 
                                                         batch_size=1000,
                                                         n_process=-1)]
textos_tratados

CPU times: user 2min 9s, sys: 960 ms, total: 2min 10s
Wall time: 5min


['polêmica marine le pen abomina negacionistas holocausto',
 'macron le pen turno frança revés siglas tradicionais',
 'apesar larga vitória legislativas macron terá desafios frente',
 'governo antecipa balanço alckmin anuncia queda homicídios sp',
 'queda maio atividade econômica sobe junho bc',
 'barcelona vence virada atlético madri bate bayern munique',
 'spartacus oferece duplo retrato batalhas perdidas',
 'sobe mortos atentado terrorista nice frança',
 'premiada sundance crystal moselle retrata sexismo mundo skate',
 'metroviários ferroviários ameaçam parar terça paulo',
 'anos angeli tirinhas diárias ilustrada',
 'mortes acidentes trânsito caem cidade sp',
 'daniel craig será stormtrooper star wars ator',
 'justiça sp libera construção moradias manancial billings',
 'prefiro urnas julguem lula achem batom cueca',
 'olivier anquier transformar cobertura centro restaurante',
 'mvp nba temporadas steve nash anuncia aposentadoria',
 'frança nega vinculo terrorista ataque soldados mes

In [28]:
df_titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
df_titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [30]:
print(len(df_titulos_tratados))
df_titulos_tratados = df_titulos_tratados.dropna().drop_duplicates()
print(len(df_titulos_tratados))

90000
84467


In [31]:
lista_lista_tokens = [titulo.split(" ") for titulo in df_titulos_tratados.titulo]

In [70]:
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

class LossPrinterCallback(CallbackAny2Vec):
  def __init__(self):
    self.epoch = 0
    self.loss_previous_step = 0
  
  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    if self.epoch == 0:
      print(f"Loss após a época {self.epoch}: {loss}")
    else:
      print(f"Loss após a época {self.epoch}: {loss}")
    self.epoch += 1
    self.loss_previous_step = loss

# sg == 0 -> CBOW
# sg == 1 -> Skip-Gram
w2v_modelo = Word2Vec(sg=0, # CBOW
                      window=2,
                      vector_size=300,
                      min_count=5,
                      alpha=0.03,
                      min_alpha=0.007,
                      compute_loss=True)
w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

In [71]:
w2v_modelo.corpus_count

84467

In [72]:
w2v_modelo.train(lista_lista_tokens, 
                 total_examples=w2v_modelo.corpus_count,
                 epochs=30,
                 callbacks=[LossPrinterCallback()])

Loss após a época 0: 0.0
Loss após a época 1: 0.0
Loss após a época 2: 0.0
Loss após a época 3: 0.0
Loss após a época 4: 0.0
Loss após a época 5: 0.0
Loss após a época 6: 0.0
Loss após a época 7: 0.0
Loss após a época 8: 0.0
Loss após a época 9: 0.0
Loss após a época 10: 0.0
Loss após a época 11: 0.0
Loss após a época 12: 0.0
Loss após a época 13: 0.0
Loss após a época 14: 0.0
Loss após a época 15: 0.0
Loss após a época 16: 0.0
Loss após a época 17: 0.0
Loss após a época 18: 0.0
Loss após a época 19: 0.0
Loss após a época 20: 0.0
Loss após a época 21: 0.0
Loss após a época 22: 0.0
Loss após a época 23: 0.0
Loss após a época 24: 0.0
Loss após a época 25: 0.0
Loss após a época 26: 0.0
Loss após a época 27: 0.0
Loss após a época 28: 0.0
Loss após a época 29: 0.0


(14584748, 16207290)

In [73]:
w2v_modelo.wv.most_similar("google")

[('apple', 0.5803342461585999),
 ('facebook', 0.523070752620697),
 ('amazon', 0.4783187210559845),
 ('tesla', 0.467632532119751),
 ('airbnb', 0.46067577600479126),
 ('uber', 0.4569011926651001),
 ('software', 0.4511861503124237),
 ('yahoo', 0.44767510890960693),
 ('news', 0.44306471943855286),
 ('sony', 0.43277955055236816)]

In [74]:
w2v_modelo.wv.most_similar("microsoft")

[('unilever', 0.5689483880996704),
 ('amazon', 0.5252668857574463),
 ('spotify', 0.5121902227401733),
 ('braskem', 0.4998159110546112),
 ('canais', 0.4972003996372223),
 ('tesla', 0.4913753569126129),
 ('viajante', 0.4855427145957947),
 ('lego', 0.48465925455093384),
 ('sony', 0.4816015362739563),
 ('edições', 0.45361068844795227)]

In [75]:
w2v_modelo.wv.most_similar("barcelona")

[('bayern', 0.5556474328041077),
 ('barça', 0.5429335236549377),
 ('chelsea', 0.5253135561943054),
 ('madrid', 0.5170454382896423),
 ('united', 0.5121796727180481),
 ('psg', 0.5115708708763123),
 ('munique', 0.5104071497917175),
 ('mourinho', 0.503070056438446),
 ('juventus', 0.4980230927467346),
 ('lazio', 0.4977955222129822)]

In [76]:
w2v_modelo.wv.most_similar("boxe")

[('canoagem', 0.6048121452331543),
 ('rúgbi', 0.590907096862793),
 ('taekwondo', 0.5796389579772949),
 ('handebol', 0.5791224241256714),
 ('duplas', 0.5637231469154358),
 ('judô', 0.5619280934333801),
 ('esgrima', 0.549457848072052),
 ('fiji', 0.5486482381820679),
 ('triatlo', 0.5308200120925903),
 ('poliana', 0.5252306461334229)]

In [77]:
w2v_modelo_sg = Word2Vec(sg=1, # Skip-Gram
                         window=2,
                         vector_size=300,
                         min_count=5,
                         alpha=0.03,
                         min_alpha=0.007,
                         compute_loss=True)
w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)
w2v_modelo_sg.train(lista_lista_tokens, 
                    total_examples=w2v_modelo.corpus_count,
                    epochs=30,
                    callbacks=[LossPrinterCallback()])

Loss após a época 0: 0.0
Loss após a época 1: 0.0
Loss após a época 2: 0.0
Loss após a época 3: 0.0
Loss após a época 4: 0.0
Loss após a época 5: 0.0
Loss após a época 6: 0.0
Loss após a época 7: 0.0
Loss após a época 8: 0.0
Loss após a época 9: 0.0
Loss após a época 10: 0.0
Loss após a época 11: 0.0
Loss após a época 12: 0.0
Loss após a época 13: 0.0
Loss após a época 14: 0.0
Loss após a época 15: 0.0
Loss após a época 16: 0.0
Loss após a época 17: 0.0
Loss após a época 18: 0.0
Loss após a época 19: 0.0
Loss após a época 20: 0.0
Loss após a época 21: 0.0
Loss após a época 22: 0.0
Loss após a época 23: 0.0
Loss após a época 24: 0.0
Loss após a época 25: 0.0
Loss após a época 26: 0.0
Loss após a época 27: 0.0
Loss após a época 28: 0.0
Loss após a época 29: 0.0


(14584320, 16207290)

In [78]:
w2v_modelo_sg.wv.most_similar("google")

[('waze', 0.43154364824295044),
 ('reguladores', 0.4217996597290039),
 ('apple', 0.421257346868515),
 ('facebook', 0.42067137360572815),
 ('exxon', 0.41168245673179626),
 ('toshiba', 0.39374449849128723),
 ('snapchat', 0.3850712478160858),
 ('monsanto', 0.3802107572555542),
 ('verizon', 0.3796663284301758),
 ('walmart', 0.3759676516056061)]

In [79]:
w2v_modelo_sg.wv.most_similar("gm")

[('metalúrgicos', 0.618873119354248),
 ('volks', 0.5789728760719299),
 ('chrysler', 0.5740767121315002),
 ('csn', 0.5697442293167114),
 ('fiat', 0.5673325657844543),
 ('honda', 0.5613058805465698),
 ('toyota', 0.5596768260002136),
 ('cubatão', 0.5447432398796082),
 ('excedente', 0.541062593460083),
 ('bp', 0.5403484106063843)]

In [81]:
w2v_modelo.wv.most_similar("gm")

[('embraer', 0.6474800109863281),
 ('volks', 0.6422678828239441),
 ('braskem', 0.6396070718765259),
 ('chrysler', 0.6378303170204163),
 ('tesla', 0.6172362565994263),
 ('volkswagen', 0.601786196231842),
 ('honda', 0.6006880402565002),
 ('toyota', 0.5977849960327148),
 ('sony', 0.5918664336204529),
 ('renault', 0.5793024301528931)]

In [82]:
w2v_modelo.wv.save_word2vec_format("/content/drive/MyDrive/ml-dataset/word2vec2/modelo_cbow.txt",
                                   binary=False)

In [83]:
w2v_modelo_sg.wv.save_word2vec_format("/content/drive/MyDrive/ml-dataset/word2vec2/modelo_skipgram.txt",
                                      binary=False)

In [91]:
import spacy
import numpy as np
from gensim.models import KeyedVectors

nlp = spacy.load("pt_core_news_sm", disable=["parser", "ner", "tagger", "textcat"])
w2v_modelo_cbow = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/ml-dataset/word2vec2/modelo_cbow.txt")
w2v_modelo_skipgram = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/ml-dataset/word2vec2/modelo_skipgram.txt")

In [92]:
def tokenizador(texto):
  doc = nlp(texto)
  tokens = []

  for token in doc:
    eh_valido = not token.is_stop and token.is_alpha
    if eh_valido:
      tokens.append(token.text.lower())

  return tokens

def combinacao_vetores_por_soma(palavras, modelo):
  vetor_resultante = np.zeros(300)
  for token in palavras:
    try:
      vetor_resultante += modelo.get_vector(token)
    except KeyError:
      pass

  return vetor_resultante

def get_matriz(textos, modelo):
  x = len(textos)
  y = 300
  matriz = np.zeros((x, y))
  for i in range(x):
    palavras = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_vetores_por_soma(palavras, modelo)
  return matriz

In [90]:
tokens = tokenizador("Rio de Janeiro 12314312 ******** @#$ é uma cidade linda!")
print(tokens)

['rio', 'janeiro', 'cidade', 'linda']


In [95]:
vetor_texto = combinacao_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(300,)
[ 5.13054579e-02  7.19350576e-03  6.72219116e-02 -1.58674987e+00
  5.74549020e-01  1.32542326e+00 -1.57522754e+00  3.74815673e-01
  6.92363519e-01 -1.75550605e+00 -2.78070397e-01  1.15573588e+00
 -7.95041338e-01 -1.49040878e-01  2.70537119e-02 -2.50820536e+00
 -9.73513037e-01 -6.59038484e-01 -1.51715502e-01  2.72062382e-01
 -3.10715377e-01 -1.05958773e+00 -8.03802423e-02  8.93911792e-01
 -1.21215641e-01 -1.18663660e+00  8.22006881e-01 -2.33630363e-01
  1.76654570e-01  8.03269535e-01 -4.99088332e-01 -2.45244215e+00
 -1.95457451e-01  1.75627010e+00  1.52922887e+00 -5.59322208e-01
 -1.59591080e+00 -1.98021813e+00  1.16661733e+00 -4.61944148e-01
 -5.25807045e-01  7.17933692e-01  9.70833503e-01 -6.81076908e-01
 -2.75706476e+00 -5.10843001e-01  1.85745910e+00 -1.00655728e+00
  1.99195087e+00 -5.01448214e-02 -4.71750705e-01 -2.97135030e+00
  7.74533149e-01 -2.33750699e+00 -3.14629041e-01  1.42121722e+00
  1.46278271e+00 -1.13704277e+00  7.81431183e-01  6.44952543e-01
 -1.41763126e+00 -

In [96]:
matriz_treino_cbow = get_matriz(df_treino.title, w2v_modelo_cbow)
matriz_teste_cbow = get_matriz(df_teste.title, w2v_modelo_cbow)

matriz_treino_skipgram = get_matriz(df_treino.title, w2v_modelo_skipgram)
matriz_teste_skipgram = get_matriz(df_teste.title, w2v_modelo_skipgram)

print((matriz_treino_cbow.shape, matriz_teste_cbow.shape))
print((matriz_treino_skipgram.shape, matriz_teste_skipgram.shape))

((90000, 300), (20513, 300))
((90000, 300), (20513, 300))


In [97]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):
  modelo_lr = LogisticRegression(max_iter=300)
  modelo_lr.fit(x_treino, y_treino)
  categorias = modelo_lr.predict(x_teste)
  resultados = classification_report(y_teste, categorias)
  print(resultados)
  return modelo_lr

cbow_lr = classificador(w2v_modelo_cbow, 
                        matriz_treino_cbow, 
                        df_treino.category,
                        matriz_teste_cbow,
                        df_teste.category)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.92      0.86      0.89      4663
   ilustrada       0.13      0.87      0.23       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [98]:
skipgram_lr = classificador(w2v_modelo_skipgram, 
                           matriz_treino_skipgram, 
                           df_treino.category,
                           matriz_teste_skipgram,
                           df_teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.87      0.90      4663
   ilustrada       0.13      0.86      0.23       131
     mercado       0.83      0.79      0.81      5867
       mundo       0.74      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [99]:
import pickle

with open("/content/drive/MyDrive/ml-dataset/word2vec2/cbow_lr.pkl", "wb") as f:
  pickle.dump(cbow_lr, f)

with open("/content/drive/MyDrive/ml-dataset/word2vec2/skipgram_lr.pkl", "wb") as f:
  pickle.dump(skipgram_lr, f)